In [39]:
import pandas as pd
import numpy as np
from prediction.functions import aggregate_prediction_results

In [253]:
models = ['logit', 'ridge', 'lasso', 'enet', 'random_forest', 'lgb', 'nn3']
models2 = ['LOGIT', 'RIDGE', 'LASSO', 'ENET', 'RF', 'GB', 'NN3']
fs_methods = ['all', 'sfi', 'mdi', 'mda', 'granger', 'huang', 'IAMB', 'MMMB']
fs_methods2 = ['ALL', 'SFI', 'MDI', 'MDA', 'GRANGER', 'HUANG', 'IAMB', 'MMMB']
tickers = ['SPX Index', 'CCMP Index','RTY Index', 'SPX Basic Materials',
           'SPX Communications', 'SPX Consumer Cyclical',
           'SPX Consumer Non cyclical', 'SPX Energy', 'SPX Financial',
           'SPX Industrial', 'SPX Technology', 'SPX Utilities']
benchmark_name = 'return_direction'
metric = "auc"
oos_start_date = '2012-07-03'
oos_end_date = '2020-12-31'

In [254]:
oos_melt_predictions_df, oos_melt_benchmark_df, oos_melt_auc_df = aggregate_prediction_results(prediction_models=models,
                                                                                               fs_models=fs_methods,
                                                                                               evaluation_start_date=oos_start_date,
                                                                                               evaluation_end_date=oos_end_date,
                                                                                               ticker_names=tickers,
                                                                                               metric_name=metric,
                                                                                               tag='oos',
                                                                                               benchmark_name=benchmark_name)

/Users/dcuoliveira/Documents/Daniel/causal_feature/src/prediction/functions.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df['model'] = benchmark_df['ticker']
/Users/dcuoliveira/Documents/Daniel/causal_feature/src/prediction/functions.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df['fs'] = 'raw'


In [255]:
oos_melt_predictions_df.head()

,date,variable,value,model,fs,ticker
2139,2012-07-03,prediction,0.887274,LOGIT,ALL,SPX Index
2140,2012-07-05,prediction,0.000002,LOGIT,ALL,SPX Index
2141,2012-07-06,prediction,1.000000,LOGIT,ALL,SPX Index
2142,2012-07-09,prediction,0.000048,LOGIT,ALL,SPX Index
2143,2012-07-10,prediction,1.000000,LOGIT,ALL,SPX Index


In [256]:
model_oos_predictions_df = oos_melt_predictions_df.loc[(oos_melt_predictions_df['variable'] == 'prediction')&
                                                       (oos_melt_predictions_df['model'] == 'RF')&
                                                       (oos_melt_predictions_df['ticker'] == 'SPX Index')]
model_oos_predictions_df.head()

,date,variable,value,model,fs,ticker
2139,2012-07-03,prediction,0.526191,RF,ALL,SPX Index
2140,2012-07-05,prediction,0.605318,RF,ALL,SPX Index
2141,2012-07-06,prediction,0.590697,RF,ALL,SPX Index
2142,2012-07-09,prediction,0.596785,RF,ALL,SPX Index
2143,2012-07-10,prediction,0.545900,RF,ALL,SPX Index


In [257]:
melt_target_df = oos_melt_benchmark_df.loc[(oos_melt_benchmark_df['variable'] == 'return_direction')&
                                           (oos_melt_benchmark_df['model'] == 'SPX Index')&
                                           (oos_melt_benchmark_df['ticker'] == 'SPX Index')&
                                           (oos_melt_benchmark_df['fs'] == 'raw')]
melt_target_df.head()

,date,variable,value,model,fs,ticker
0,2012-07-03,return_direction,1.0,SPX Index,raw,SPX Index
1,2012-07-05,return_direction,0.0,SPX Index,raw,SPX Index
2,2012-07-06,return_direction,0.0,SPX Index,raw,SPX Index
3,2012-07-09,return_direction,0.0,SPX Index,raw,SPX Index
4,2012-07-10,return_direction,0.0,SPX Index,raw,SPX Index


## 1. Aggregating by forecasting model (across feature selection methods)

In [270]:
cov_e

matrix([[1.01565917e-06, 4.74392190e-04, 4.85877810e-04, 5.03928158e-04,
         1.03395732e-06, 1.00779917e-06, 4.78117919e-04, 1.00779917e-06],
        [4.74392190e-04, 2.21578220e-01, 2.26942902e-01, 2.35373825e-01,
         4.82938860e-04, 4.70720958e-04, 2.23318427e-01, 4.70720958e-04],
        [4.85877810e-04, 2.26942902e-01, 2.32437470e-01, 2.41072515e-01,
         4.94631406e-04, 4.82117693e-04, 2.28725241e-01, 4.82117693e-04],
        [5.03928158e-04, 2.35373825e-01, 2.41072515e-01, 2.50028353e-01,
         5.13006949e-04, 5.00028352e-04, 2.37222378e-01, 5.00028352e-04],
        [1.03395732e-06, 4.82938860e-04, 4.94631406e-04, 5.13006949e-04,
         1.05258514e-06, 1.02595572e-06, 4.86731712e-04, 1.02595572e-06],
        [1.00779917e-06, 4.70720958e-04, 4.82117693e-04, 5.00028352e-04,
         1.02595572e-06, 1.00000000e-06, 4.74417854e-04, 1.00000000e-06],
        [4.78117919e-04, 2.23318427e-01, 2.28725241e-01, 2.37222378e-01,
         4.86731712e-04, 4.74417854e-04, 2.25

In [268]:
oos_melt_predictions_df, oos_melt_benchmark_df, oos_melt_auc_df = aggregate_prediction_results(prediction_models=models,
                                                                                               fs_models=fs_methods,
                                                                                               evaluation_start_date=oos_start_date,
                                                                                               evaluation_end_date=oos_end_date,
                                                                                               ticker_names=tickers,
                                                                                               metric_name=metric,
                                                                                               tag='oos',
                                                                                               benchmark_name=benchmark_name)

forecast_star_list = []
for model in models2:
    for ticker in tickers:
        model_oos_predictions_df = oos_melt_predictions_df.loc[(oos_melt_predictions_df['variable'] == 'prediction')&
                                                               (oos_melt_predictions_df['model'] == model)&
                                                               (oos_melt_predictions_df['ticker'] == ticker)]

        melt_target_df = oos_melt_benchmark_df.loc[(oos_melt_benchmark_df['variable'] == 'return_direction')&
                                                   (oos_melt_benchmark_df['model'] == ticker)&
                                                   (oos_melt_benchmark_df['ticker'] == ticker)&
                                                   (oos_melt_benchmark_df['fs'] == 'raw')]

        forecast_df = model_oos_predictions_df.pivot_table(index=['date'], columns=['model', 'fs'], values=['value']) - 0.001
        target_df = melt_target_df.pivot_table(index=['date'], columns=['model'], values=['value'])

        l_m = np.matrix(np.ones(shape=resid_df.shape[1]))

        f_star_list = []
        for i in range(forecast_df.shape[0]):
            e = np.matrix(l_m.T * target_df.values[i] - np.matrix(forecast_df.values[i]).T)
            cov_e = e.dot(e.T)
            w_star = np.linalg.inv(l_m.dot(np.linalg.inv(cov_e)).dot(l_m.T)).item() * np.linalg.inv(cov_e).dot(l_m.T) 

            f_star = forecast_df.values[i].dot(w_star).item()
            f_star_list.append(np.abs(np.round(f_star, 0)))
        
        forecast_df = pd.DataFrame(f_star_list,
                                   columns=[model + '_star'],
                                   index=target_df.index)
        forecast_star_list.append(forecast_df)

KeyboardInterrupt: 